# Codigo del proyecto

## 1. Deficion del problema


### - Objetivo: Predecir la probabilidad de cancelación que tienen los clientes.
#### Métricas objetivo:
- Métrica principal: AUC-ROC (Área bajo la curva de la curva ROC).
- Métrica adicional: Exactitud.
#### Criterios de evaluación:
- AUC-ROC < 0.75: 0 SP.
- 0.75 ≤ AUC-ROC < 0.81: 4 SP.
- 0.81 ≤ AUC-ROC < 0.85: 4.5 SP.
- 0.85 ≤ AUC-ROC < 0.87: 5 SP.
- 0.87 ≤ AUC-ROC < 0.88: 5.5 SP.
- AUC-ROC 


## 1.1 Carga de los datos≥ 0.88: 6 SP.

In [ ]:
# Importar librerias y modulos requeridos para el ejecucion del codigo

# Librerias para manejo de dataframe y arreglos 
import pandas as pd
import numpy as np

# Modulos para mejorar la salida de los datos en pantalla
from IPython.display import display
from tqdm import tqdm
import time

# Modulos para graficar
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Librerias y modulos para ML
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, log_loss
import shap

In [ ]:
# Cargar los conjuntos de datos para la realicion del proyecto

# Lista para guardar el nombre de los dataframe
name_df = []

# Dataframe que tiene la informacion personal de los clientes
df_personal = pd.read_csv('../Datasets/personal.csv')
name_df.append('df_personal')

# Dataframe que tiene la informacion de los contratos
df_contract = pd.read_csv('../Datasets/contract.csv')
name_df.append('df_contract')

# Dataframe que tiene la informacion de los servicios de internet
df_internet = pd.read_csv('../Datasets/internet.csv')
name_df.append('df_internet')

# Dataframe que tiene la informacion de los servicios telefonicos
df_phone = pd.read_csv('../Datasets/phone.csv')
name_df.append('df_phone')    

Verificacion de la carga y estructura de los conjuntos de datos. 

In [ ]:
# Listar la informacion de los dataframe para validar que se hallan cargado satisfactoriamente y poder analizar su estructura

# Recorrer la lista de nombre de los dataframe para ver su estructura
for df_name in name_df:
    print(f'Conjunto de datos con la informacion de {df_name}')
    print('-' * 60)
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    display(dataframe.info())
    print('')

## 2. Entendimiento del Conjunto de Datos


### Revisión inicial:
- Analizar la columna EndDate para confirmar que los clientes con valor ***"No"*** están etiquetados como no cancelados.
- Identificar el balance de clases (proporción de cancelados vs. no cancelados).
#### Preguntas clave:
- ¿Qué variables parecen correlacionarse más con la cancelación?

- ¿Hay variables redundantes o irrelevantes que se puedan 


## 2.1 Revisicion inicial de los datosminar?   

In [ ]:
# Revisar los valores de las columna EndDate
print('Valores unicos en la columna EndDate')
display(df_contract['EndDate'].unique())
print('-' * 70)

# Se filtran los clientes activos
active_customer = (df_contract['EndDate'] == 'No')
total_active_customer = active_customer.sum()

# Se hallan el total de clientes en el conjunto de datos
total_customer = len(df_contract['EndDate'])

# Se imprime el numero y participacion de clientes activos y los que cancelaro su contrato
print(f'Total clientes activos: {total_active_customer} --> {total_active_customer / total_customer * 100:.2f}% ')
print(f'Total clientes que cancelaron su contrato: {total_customer - total_active_customer} --> {(total_customer - total_active_customer) / total_customer * 100:.2f}%')                     

Al revisar las diferentes caracteristicas aportadas por los conjutos de datos, se considera relevante analizar la influencia de las siguientes columnas en las predicciones del modelo:
 
    df_personal
        - gender
        - SeniorCitizen
 
    df_contract
        - PaymentMethod
        - TotalCharges

    df_internet
        - TechSupport
 
    df_phone
        - MultipleLines

***Analisis datos columna EndDate***

- Valores unicos: los valores unicos no presentan inconsistencias o rangos no permitidos.
- Balance de clases: el ***73.46%*** de los clientes tienen su contrato vigente y el ***26.54%*** lo han cancelado, esto hace que posiblemente de deba usar sobremuestreo en el conjunto de datos para poder llevar a cabo un entrenamiento sin sesgos.

No se observan columnas con datos irrelevantes, sin embargo se deben hacer pruebas de incidencia en los resultados para poder determinar la relevancia de caracteristicas y asi depurar los datos para una mejor interpretacion p

## 3. Preparacion de los datos
<a id='preparacion_de_los_datos'></a>

Durante la preparacion de los datos se llevaran a cabo las siguientes tareas:

- Limpieza de datos.
- Codificación y transformación.
- Análisis exploratorio de datos (EDA).

### 3.1 Limpieza de datos


- Se pasan a minuscula los nombres de las columnas.
- Se pasa a minuscula el contenido de las columnas de tipo object.
- Se eliminan espacios en blanco al inicio y al final de cada dato.
- Se buscan registros duplicados.
- Se revisan valores ausentes.
- Se revisan los valores minimo y maximo de cada columna.
- Se revisan los valores unicos de cada columna.
- Se revisan y cambian los tipos de datos segun sea necesario para tratar la informacion.   (EDA)
or parte del modelo.


In [ ]:
# Pasar a minuscula todas las columnas de datos tipo object

# Se recorre la lista de nombre de los dataframe
for df_name in name_df:
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    dataframe.columns = dataframe.columns.str.lower() # Pasa a minuscula el nombre de las columnas del dataframe
    for column in dataframe.columns: # Recorre las columnas del dataframe
        if dataframe[column].dtype == 'object': # Valida que la columna sea de tipo object
           dataframe[column] = dataframe[column].str.lower() # Pasa a minuscula todo el contenido de la columna
           dataframe[column] = dataframe[column].str.strip() # Elimina espacios en blanco al inicio y al final de cada dato
           
# Se imprime una muestra aleatoria de cada conjunto de datos para validar que el cambio se halla efectuado satisfactoriamente
for df_name in name_df:
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    print(f'Conjunto de datos {df_name}')
    print('')
    display(dataframe.head())
    print('-' * 60)
    print('')

In [ ]:
# Cambiar los valores de yes a 1 y de no a 0 en las columnas partner, dependents paperlessbilling y multiplelines
df_personal['partner'] = df_personal['partner'].map({'yes':'1', 'no':'0'})
df_personal['dependents'] = df_personal['dependents'].map({'yes':'1', 'no':'0'})
df_contract['paperlessbilling'] = df_contract['paperlessbilling'].map({'yes':'1', 'no':'0'})
df_phone['multiplelines'] = df_phone['multiplelines'].map({'yes':'1', 'no':'0'})

In [ ]:
# Buscar registros duplicados

for df_name in name_df:
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    print(f'Conjunto de datos {df_name}')
    duplicated_row = dataframe.duplicated().sum()
    print(f'Numero de filas duplicadas: {duplicated_row}')
    print('-' * 30)
    print('')

No se hallan registros duplicados en los conjuntos de datos.

In [ ]:
# Revisar valores ausentes

for df_name in name_df:
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    print(f'Conjunto de datos {df_name}')
    for column in dataframe.columns: # Recorre las columnas del dataframe
        print('Columna: ', column)
        print('Total valores ausentes: ', dataframe[column].isna().sum())
        total_isna = dataframe[column].isna().sum()
        len_column_dataframe = len(dataframe[column])
        print(f'Porcentaje valores ausentes: {total_isna / len_column_dataframe * 100:.2f}%')
        print('-' * 30)

    print('*' * 30)
    print('')

***Analisis valores ausentes***

Solo se encontraron valores ausentes en dos de las columnas de los cuatros conjuntos de datos, este es el detalle los hallagos:

- df_contratc

    - columna enddate: tiene 5174 valores ausentes que representa el ***73.46%*** de los datos. Estos valores ausentes existen porque la columna hace referencia a la fecha de cancelacion del contrato y al estar vigente el contrato, no existe fecha de cancelacion.

    - columna totalcharges: tienee 11 valores ausentes que representan el ***0.16%*** de los datos. Por ser tan poco representativa la cantidad de registros con valores ausentes, estos se van a eliminar.

In [ ]:
# Eliminar los valores ausentes en la columna totalcharges del conjunto de datos df_contract

df_contract['totalcharges'] = df_contract['totalcharges'].dropna()

print('valores ausentes en la columna totalcharges: ', df_contract['totalcharges'].isna().sum())

In [ ]:
# Revisar los valores minimo y maximo de las columnas numericas

for df_name in name_df:
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    print(f'Conjunto de datos {df_name}')
    for column in dataframe.columns: # Recorre las columnas del dataframe
        print('Columna: ', column)
        print('Tipo de dato: ', dataframe[column].dtype)
        min_value = dataframe[column].min()
        max_value = dataframe[column].max()
        print(f'Valor minimo: {min_value}')
        print(f'Valor minimo: {max_value}')
        print('-' * 30)

    print('*' * 30)
    print('')

***Analisis valores maximo y minimo***

Al revisar los valores minimo y maximo de todos los conjuntos de datos, se pueden observar las siguientes novedades:

- Conjunto de datos df_personal:
    - La columna seniorcitizen maneja valores de 0 y 1 con un tipo de dato object. Se pasara int para disminuir el consumo de memoria al almecenar el dato.

- Conjunto de datos df_contract
    - La columna begindate maneja fechas con un tipo de dato object. Se pasara a tipo datetime para procesar correctamente los datos de fecha.
    - La columna enddate maneja fechas con un tipo de dato object. Se pasara a tipo datetime para procesar correctamente los datos de fecha.
    - La columna totalcharges maneja datos numericos con un tipo de dato objetc. Se pasara a float64 para procesar correctamente los datos.

Las demas columnas tiene un tipo de dato acorde a la informacion que representan.

In [ ]:
# Revisar los valores unicos de cada columna

for df_name in name_df:
    # Acceder al dataframe por su nombre
    dataframe = globals()[df_name]
    print(f'Conjunto de datos {df_name}')
    for column in dataframe.columns: # Recorre las columnas del dataframe
        print('Columna: ', column)
        print('Valores unicos: ', dataframe[column].unique())
        print('-' * 30)

    print('*' * 30)
    print('')

***Analisis valores unicos***

Los valores unicos encontrados en las diferentes columnas estan dentro de los valores permitidos segun la informacion de representan, sin embargo, la columna begindate del conjunto de datos df_contract maneja fechas, estos datos estan estructurados como object; la columna de debe pasar a datetime para que las fechas se procesen adecuadamente.

In [ ]:
# Cambiar el tipo de dato en las columnas que se mencionan en el analisis de los valores minimo y maximo

df_personal['seniorcitizen'] = df_personal['seniorcitizen'].astype('int64')
df_contract['begindate'] = pd.to_datetime(df_contract['begindate'])

In [ ]:
# Revisar los datos de la columna totalcharges para saber si todos se pueden convertir a tipo float

# Se reemplaznr valores no numéricos con NaN temporalmente
non_numeric = pd.to_numeric(df_contract['totalcharges'], errors='coerce').isna()

# Se cuentan filas con valores no numéricos
non_numeric_count = non_numeric.sum()

print(f"Filas con valores no numéricos: {non_numeric_count}")

# Eliminar filas con valores no numéricos en 'totalcharges'
df_contract = df_contract[~non_numeric]

df_contract['totalcharges'] = df_contract['totalcharges'].astype('float64')

Al encontrar solo 11 filas con valores no numericos en la columna totalcharger, estos registros se eliminan.

Se procede a verificar que los cambios de tipo de dato se hallan efectuado satisfactoriamente.

In [ ]:
# Imprimir 5 filas aleatorias y los valores unicos para cada columna

print('Conjunto de datos df_personal')
display(df_personal['seniorcitizen'].sample(5))
print('Valores unicos', df_personal['seniorcitizen'].unique())
print('-' * 60)
print('')

print('Conjunto de datos df_contract')
display(df_contract['begindate'].sample(5))
print('Valores unicos', df_contract['begindate'].unique())
print('-' * 60)
print('')

print('Conjunto de datos df_contract')
display(df_contract['enddate'].sample(5))
print('Valores unicos', df_contract['enddate'].unique())
print('-' * 60)
print('')

print('Conjunto de datos df_contract')
display(df_contract['totalcharges'].sample(5))
print('Valores unicos', df_contract['totalcharges'].unique())
print('-' * 60)
print('')

### 3.1.1 Codificación y transformación

Se realizaran las siguientes tareas:

    - Consolidar los datos en un único registro por cliente.
        - Revisar valores ausentes en el nuevo conjunto de datos.
    - Codificar variables categóricas, como métodos de pago y tipos de contrato, utilizando técnicas como One-Hot Encoding.
    - Escalar variables numéricas para modelos sensibles a la escala.
    - Crear nuevas características útiles, como duración del contrato en días o número total de servicios cont


***Funcion para unir los conjuntos de datos***ratados.

In [ ]:
# Generar un unico conjunto de datos con un registro por cliente

df_consolidated = pd.DataFrame()

def join_df(df_left, df_right):
    """
    La funcion join_df une dos dataframe conservando los registros del dataframe de la izquierda y usando como clave la columna customerid

    Parametros
    df_left: pasa el dataframe que se usara a la izquierda de la union.
    df_right: pasa el dataframe que se usara a la izquierda de la union.

    Retorno
    El retorno de la funcion en en nuevo dataframe con el mismo numero de filas del conjunto de datos de la izquierda y la suma de las columnas de los
    dos conjuntos de datos menos una, que es la que se usa como clave de la union.
    
    """
    total_columns_left = df_left.shape[1]
    total_columns_right = df_right.shape[1]
    
    total_columns_consolidated = total_columns_left + total_columns_right - 1
    total_rows_left = len(df_left)
    
    print('Numero de columnas del conjunto de datos de la izquierda: ', total_columns_left)
    print('Numero de filas del conjunto de datos de la izquierda', len(df_left))
    print('')
    print('Numero de columnas del conjunto de datos de la derecha: ', total_columns_right)
    print('Numero de filas del conjunto de datos de la derecha', len(df_right)) 
    print('')
    df_join = pd.merge(df_left, df_right, on='customerid', how='left')
    print('Numero de columnas del nuevo conjunto de datos: ', total_columns_consolidated)
    print('Numero de filas del conjunto de datos de la derecha', len(df_join))
    print('')

    return df_join, total_columns_consolidated, total_rows_left

In [ ]:
# Funcion validar el tamaño de la fusion

def validation():
    """
    La funcion valida el tamaño del nuevo conjunto de datos al comparar el numero de filas y columnas resultantes con los datos 
    generados antes de la fusion

    Parametros
    La funcion no utiliza parametros

    Retorno
    La funcion no genera un retorno, unicamente muestra en pantalla si el nuevo conjunto de datos cumple o no con el numero de filas y columnas esperados
    """
    print('Validacion de la fusion')
    print('-' * 80)
    
    columns = df_consolidated.shape[1]
    rows = len(df_consolidated)
    
    if columns_validation ==  columns and rows == rows_validation: 
        print('El nuevo conjunto de datos cumple con el numero de filas y columnas esperados')
        print('-' * 80)
        print('Numero de columnas del nuevo conjunto de datos: ', df_consolidated.shape[1])
        print('Numero de filas del nuevo conjunto de datos: ',len(df_consolidated))
    else:
        print('El nuevo conjunto de datos no cumple con el numero de filas y columnas esperados')

***Union de los conjuntos de datos df_personal y contract***

In [ ]:
# Llamar a la funcion
df_consolidated, columns_validation, rows_validation = join_df(df_personal, df_contract)

validation()

***Union de los conjuntos de datos df_consolidated y df_internet***

In [ ]:
# llamar a la funcion
df_consolidated, columns_validation, rows_validation = join_df(df_consolidated, df_internet)

validation()

***Union de los conjuntos de datos df_consolidated y df_phone***

In [ ]:
# Llamar a la funcion
df_consolidated, columns_validation, rows_validation = join_df(df_consolidated, df_phone)

validation()

Revisar valores ausente en el nuevo conjunto de datos

In [ ]:
display(df_consolidated.isnull().sum())

Se procede a validar el tamaño de los conjunto de datos para hallar las diferencias de filas con el conjunto de datos df_personal y validar si los valores ausentes en el nuevo conjunto de datos df_consolidated corresponden a esta diferencia.

In [ ]:
len_df_personal = len(df_personal)
len_df_contract = len(df_contract)
len_df_internet = len(df_internet)
len_df_phone = len(df_phone)

print('Numero de filas del conjunto de datos de df_personal: ', len_df_personal)
print('Numero de filas de diferencia del conjunto de datos df_personal y de df_contract: ', len_df_personal - len_df_contract)
print('Numero de filas de diferencia del conjunto de datos df_personal y de df_internet: ', len_df_personal - len_df_internet)
print('Numero de filas de diferencia del conjunto de datos df_personal y de df_phone: ', len_df_personal - len_df_phone)

La validacion confirma que los valores ausentes en el nuevo conjunto de datos df_consolidated corresponden a las diferencias del numero de filas. 
Conociendo esto, se procede a reemplazar los valores ausentes

Se obtienen los nombres de las columnas que tiene valores ausentes en el nuevo conjunto de datos df_consolidated..

In [ ]:
# Crear lista para guardar el nombre de las columnas con valores ausentes
name_columns_na = []

# Se recorren todas las columnas de dataframe
for column in df_consolidated.columns:
    values_na = df_consolidated[column].isnull().sum()
    if values_na > 0:
        name_columns_na.append(column)

display(name_columns_na)

Se gerenan los valores unicos de las columnas con valores ausentes para determinar con que valor se pueden reemplazar estos.

In [ ]:
for column in range(len(name_columns_na)):
    column_na = name_columns_na[column]
    print('Valores unicos en la columna ', column_na)
    display(df_consolidated[column_na].unique())
    print('-' * 80)
    print('')

Para tratar as columnas con valores ausentes, se procesaran primero las que menos datos tengan por reemplazar para asi tener una estructura mas completa al momento de abordar las columnas con mayor cantidad de datos a ausentes.

In [ ]:
def customer_validated(df_a, df_b, column_validated):
    """
    La funcion recibe tres parametros, los dos primeros son dataframe, y con ellos se busca que elementos del primer conjunto de datos no
    se encuentra en el segundo. El tercer parametro es la columna que se usara para hacer la busqueda

    Parametros
    df_a: es el dataframe que se usa para selecionar los elementos a buscar en el segundo dataframe
    df_b: es el dataframe en el cual se hace la busqueda de los valores.
    column_validated: es el nombre de la columna que se usara para poder hacer la busqueda

    Retorno
    La funcion no retorna valores, pero genera impresiones que indican los resultados obtenidos
    """
    # Se crean conjuntos con los datos de la columna customer_id de los dataframe df_personal y df_contract
    set_a = set(df_a[column_validated])
    set_b = set(df_b[column_validated])
    
    # Se hallan los datos del conjunto de df_personal que no estan en df_contract
    difference_set_a = set_a - set_b
    print('Cantidad de valores del conjunto de datos df_a que no estan en df_b: ', len(difference_set_a))
    print('Datos de df_a que no estan en df_b: ', difference_set_a)
    print('')
    
    # Se crea una lista con los datos de la columna customerid de df_a que no estan en df_b
    list_difference_set_a = list(difference_set_a)

    # S crea una lista para guardar las coincidencias
    list_coincidense = []
    
    # Se listan los regitros del conjunto de datos df_consolidated que coinciden en la columna customerid con la lista list_personal
    print('Codigos de cliente que tienen valores ausentes en df_consolidated')
    for search_for_customer in list_difference_set_a:
        row_search_for_customer = df_consolidated[df_consolidated[column_validated] == search_for_customer]
        list_coincidense.append(search_for_customer)

    print('-' * 80)
    print(list_coincidense)

In [ ]:
# Llamar a la funcion customer_validated para validar que clientes del df_personal no tiene un registro en el df_contract
customer_validated(df_personal, df_contract, 'customerid')

Al observar los registros del dataframe df_consolidated que coinciden en su columna ***customerid*** con los elementos de la lista ***list_difference_set_a*** (clientes que no tiene contrato), se puede evidenciar que estos registros presenta valores ausentes porque no tiene un contrato. Motivo por el cual se procedera a eliminar estas filas, ya que representan el 0.15% del total de los registros y, este porcentaje no afecta el resultado que entregue el modelo predictivo.

In [ ]:
# Numero de filas del conjunto de datos df_consolidated antes de eliminar los registros de los clientes que no tienen contrato
print('tamaño inicial: ', df_consolidated.shape[0])

# Se eliminan los registros de la columna begindate que tiene valores ausentes
df_consolidated = df_consolidated.dropna(subset=['begindate'])

# Numero de filas del conjunto de datos df_consolidated despues de eliminar los registros de los clientes que no tienen contrato
print('tamaño final: ', df_consolidated.shape[0])

display(df_consolidated.isna().sum())

***Analisis manejo de las columnas con valores ausentes en df_consilidated***

Columnas procesadas:

- ***begindate***: esta columna tenia valores ausentes porque pertenecian a clientes que no tenian un contrato.
- ***type***: esta columna tenia valores ausentes porque pertenecian a clientes que no tenian un contrato.
- ***paperlessbilling***: esta columna tenia valores ausentes porque pertenecian a clientes que no tenian un contrato.
- ***paymentmethod***: esta columna tenia valores ausentes porque pertenecian a clientes que no tenian un contrato.
- ***monthlycharges***: esta columna tenia valores ausentes porque pertenecian a clientes que no tenian un contrato.
- ***totalcharges***: esta columna tenia valores ausentes porque pertenecian a clientes que no tenian un contrato.

Acontinuacion de continua con el procesamiento de los valores ausentes de la columna multiplelines.

In [ ]:
# Llamar a la funcion customer_validated para validar que clientes del df_contract no tiene un registro en el df_phone
customer_validated(df_contract, df_phone, 'customerid')

Al observar los registros del dataframe df_consolidated que coinciden en su columna ***customerid*** con los elementos de la lista ***list_difference_set_a*** (clientes que no tienen contratado el servicio de telefono), se observa que son el mismo numero de filas que tiene df_consolidated en su columna multiplelines con valores ausentes.

La columna multiplelines solo maneja dos posible valores: ***yes*** y ***no***. Estos valores se reemplazaran por 1 para ***yes*** y 0 para ***no. Para los registros de df_consolidated que tiene valor ausente en la columna multiplelines, este valor se reemplazara por '***0***'

In [ ]:
# De df_consolidated de su columna multiplelines se reemplazan los valores ausentes por '0' 
df_consolidated['multiplelines'].fillna('0', inplace=True)
display(df_consolidated.isna().sum())

A continuacion de continua con el procesamiento de los valores ausentes de las siguientes columnas: internetservice, onlinesecurity, onlinebackup, deviceprotection, techsupport, streamingtv, streamingmovies.

In [ ]:
# Llamar a la funcion customer_validated para validar que clientes del df_contract no tiene un registro en el df_internet
customer_validated(df_contract, df_internet, 'customerid')

Al observar los registros del dataframe df_consolidated que coinciden en su columna ***customerid*** con los elementos de la lista ***list_difference_set_a*** (clientes que no tienen contratado el servicio de internet), se observa que son el mismo numero de filas que tiene df_consolidated en sus columnas:  internetservice, onlinesecurity, onlinebackup, deviceprotection, techsupport, streamingtv, streamingmovies con valores ausentes.

Se procede a revisar los valores unicos de las columnas con valores ausentes para determinar con que dato se debe reemplazar.

In [ ]:
# Obtener una lista con los nombres de las columnas que aun tienen valores ausentes en df_consolidated
columns_null = df_consolidated.columns[df_consolidated.isnull().any()].tolist()

for column in columns_null:
    print('Valores unicos de la columna: ', column)
    display(df_consolidated[column].unique())
    print('-' * 80)

# Se la lista de las columnas se elimina enddate, ya que esta no maneja valores yes y no 
if 'enddate' in columns_null:
    columns_null.remove('enddate')  

# Se la lista de las columnas se elimina internetservice, ya que esta no maneja valores yes y no 
if 'internetservice' in columns_null:
    columns_null.remove('internetservice')

La columna multiplelines solo maneja dos posible valores: ***yes*** y ***no***. Estos valores se reemplazaran por 1 para ***yes*** y 0 para ***no. Para los registros de df_consolidated que tiene valor ausente en la columna multiplelines, este valor se reemplazara por '***0***'


Todas las columnas manejan solo dos posibles valores, ***yes*** y ***no***. Estos valores se reemplazaran por 1 para ***yes*** y 0 para ***no***. Para los registros de df_consolidated que tiene valor ausente en las columnas internetservice, onlinesecurity, onlinebackup, deviceprotection, techsupport y streamingtv, se reemplazarn con 0.

In [ ]:
# Recorrer la lista de las columnas para buscarlas en df_consolidated y hacer el respectivo cambio de valores
for column_null in columns_null:
    df_consolidated[column_null] = df_consolidated[column_null].map({'yes':'1', 'no':'0'}) 

# Se reemplazar por 0 los valores ausentes
for column in columns_null:
    df_consolidated[column].fillna('0', inplace=True)

In [ ]:
# Cambiar los valores ausentes en la columna internetservice por no service 
df_consolidated['internetservice'].fillna('no service', inplace=True)

display(df_consolidated.isna().sum())

Al listar de nuevo los valores ausentes en df_consolidated, se puede observar que solo quedan ausentes los valores de la columna ***enddate*** y, estos valores representan a los clientes que aun tienen vigente su contrato, por este motivo no existe una fecha de finalizacion. 

Para poder tener una columna de objetivos en el dataframe, se adicionara una columna llamada ***finalizedcontract*** con valores de ***1*** para los datos que tiene una fecha asignada y ***0*** para los datos que tienen valores ausentes.

In [ ]:
# Crear la columna finalizedcontract que se usara como objetivo para el modelo
df_consolidated['finalizedcontract'] = df_consolidated['enddate'].map({'2019-12-01 00:00:00':'1', '2019-11-01 00:00:00':'1', '2020-01-01 00:00:00':'1', '2019-10-01 00:00:00':'1', 'no':'0'})
print('')

# Se revisan los valores unicos en la columna enddate
print('Valores unicos en la columna enddate')
display(df_consolidated['enddate'].unique()) 
print('')

# Se revisan los valores unicos en la columna finalizedcontract
print('Valores unicos en la columna finalizedcontract')
display(df_consolidated['finalizedcontract'].unique()) 

# Se cambia el tipo de dato de la columna para evitar posibles errores en el entrenamiento de los modelos predictivos
df_consolidated['finalizedcontract'] = df_consolidated['finalizedcontract'].astype('int')
print('')

# Se validan los valores de la nueva columna
count_0 = df_consolidated['finalizedcontract'].value_counts()
print('Numero de veces que se repiten los valores unicos en la columna finalizedcontract')
print(count_0)

***Analisis de la columna finalizedcontract***

La columna finalizedcontract que se usara como objetivo para el modelo presdictivo, tiene solo dos valores, 1 y 0. Siendo el 0 el valor que representa a los clientes que aun tiene su contrato vigente. El numero de veces que se repite el 0 en la columna es 5163, la columna tiene 11 filas menos con clientes que no tenian fecha de finalizacion. Estos 11 registros son los mismos que se eliminaron del df_personal pòr no tener un contrato asignado.

Se procede a generar de nuevo los valores unicos en df_consolidated para verificar que los cambios efectudos se hallan aplicado correctamente.

In [ ]:
# Revisar los valores unicos en las columnas del nuevo conjunto de datos.
columns = df_consolidated.columns

for column in columns:
    print(column)
    display(df_consolidated[column].unique())

Se realiza transformacion de columnas, separacion de caracteristicas y el objetivo para el entrenamiento del modelo predictivo.

In [ ]:
# Dividir la columna begindate para brindar mayor detalle al entrenamiento del modelo
df_consolidated['yearbegindate'] = df_consolidated['begindate'].dt.year
df_consolidated['mesbegindate'] = df_consolidated['begindate'].dt.month
df_consolidated['diabegindate'] = df_consolidated['begindate'].dt.day

# Las columnas mesbegindate y diabegindate se pasan a tipo objecto por tener datos con clasificacion ordinal
df_consolidated['mesbegindate'] = df_consolidated['mesbegindate'].astype('object')
df_consolidated['diabegindate'] = df_consolidated['mesbegindate'].astype('object')

# Se extraen las caracteristicas y el objetivo para el entrenamiento del modelo
features = df_consolidated.drop(columns=['customerid', 'begindate', 'enddate', 'finalizedcontract'])
target = df_consolidated['finalizedcontract']

# Se separan los nombres de las columnas segun el tipo de dato para la posterior codificacion o escalado
categorical_columns = features.select_dtypes(include=['object']).columns.tolist()
numeric_columns = features.select_dtypes(include=['float', 'int']).columns.tolist()

print('Total columnas tipo objetc: ', len(categorical_columns))
print('Total columnas tipo numerico: ',len(numeric_columns))

print('')
print('Total filas en features: ', features.shape[0])
print('Total columnas en features: ', features.shape[1])

### 3.1.2 Análisis exploratorio de datos (EDA)

Validacion de balance de clases

In [ ]:
# Se genera el conteo para cada clase en el objetivo
class_counts = target.value_counts()
    
print('Balance de clases para el objetivo: ', class_counts)
    
# Gráfico de barras para el balance de clases
sns.countplot(x=target)
plt.title("Balance de Clases en el Objetivo")
plt.xlabel("Clase")
plt.ylabel("Frecuencia")
plt.show()

Codificacion y escalado de las caracteristicas

In [ ]:
# Escaladar de las caracteristicas nunericas
scaler = StandardScaler()
columns_scaled = {}

# Se recorren las columnas numericas para usar la barra tqdm
for column in tqdm(numeric_columns, 'Escalado de olumnas numericas'):
    # Se escala una a una las columnas
    columns_scaled[column] = scaler.fit_transform(features[[column]])

# Se transforma en dataframe el resultado del escalado
df_columns_scaled = pd.DataFrame({col: columns_scaled[col].flatten() for col in numeric_columns})

# Codificacion de las columnas categoricas
encoder = OneHotEncoder(sparse_output=False)
columns_encode = []

# Se recorren las columnas numericas para usar la barra tqdm
for column in tqdm(categorical_columns, desc='Codificación de columnas categóricas'):
    encoded_column = encoder.fit_transform(features[[column]])  #  Se codifica cada columna individualmente
    column_names = [f"{column}_{cat}" for cat in encoder.categories_[0]]  # Se generan nombres únicos para las columnas
    encoded_df = pd.DataFrame(encoded_column, columns=column_names)  # Se crea un DataFrame para la columna codificada
    columns_encode.append(encoded_df)  # Agrega al listado de DataFrames codificados

# Combina todas las columnas codificadas en un solo DataFrame
df_columns_encode = pd.concat(columns_encode, axis=1)

# Se combinan los resultados
features_transform = pd.concat([df_columns_scaled, df_columns_encode], axis=1)

print('Numero de filas en el conjunto de caracteristicas despues del escalado y la codificacion: ', features_transform.shape[0])
print('Numero de columnas en el conjunto de caracteristicas despues del escalado y la codificacion: ', features_transform.shape[1])

Separacion de los conjuntos de entrenamiento y validacion

In [ ]:
# Eliminar espacios al inicio y al final, y reemplazar espacios en medio por guiones bajos
features_transform.columns = features_transform.columns.str.strip().str.replace(' ', '_')

# Se separan los conjuntos de datos para entrenammiento y validacion
features_train, features_test, target_train, target_test = train_test_split(features_transform, target, random_state=12345, test_size=0.2)

# Se verifica del tamaño de los conjuntos de entrenamiento y validacion
print('Numero de filas del conjunto de entrenamiento: ', features_train.shape[0]) 
print('Numero de columnas del conjunto de entrenamiento: ', features_train.shape[1]) 
print('Numero de filas del objetivo para el entrenamiento: ', target_train.shape[0])
print('')

print('Numero de filas del conjunto de validacion: ', features_test.shape[0])
print('Numero de columnas del conjunto de validacion: ', features_test.shape[1])
print('Numero de filas del objetivo para el entrenamiento: ', target_test.shape[0])

Sobremuestreo de clases

In [ ]:
# Generar el conteo para cada clase en el objetivo de entrenamiento
class_counts = target_train.value_counts()
print('Balance original de clases para el objetivo de entrenamiento: ', class_counts)
print('')

# Se aplica SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
features_train_resampled, target_train_resampled = smote.fit_resample(features_train, target_train)

# Se verificar la nueva distribución de clases
class_counts = target_train_resampled.value_counts()
print('Balance con sobremuestreo de clases para el objetivo de entrenamiento: ', class_counts)

## 4. Construcción del Modelo Predictivo


Funcion para evaluar el resultado del modelo

In [ ]:
def result_classification(value_metric):
    """
    La funcion recibe el resultado de la metrica obtenido para la curva ROC
    y lo clasifica segun los rangos entregados en el proyecto. Esta funcion se uitiliza en cada uno de los modelos,
    despues de haber realizado las predicciones.
    
    Parametros:
    value_metric: valor de la metrica curve roc
    
    Return
    La funcion retorna el valor de SP asignado segun el resultado de la curva ROC
    """
    if value_metric < 0.75:
        sp = 0
    elif 0.75 <= value_metric < 0.81:
        sp = 4
    elif 0.81 <= value_metric < 0.85:
        sp = 4.5
    elif 0.85 <= value_metric < 0.87:
        sp = 5
    elif 0.87 <= value_metric < 0.88:
        sp = 5.5
    else:
        sp = 6

    return sp

Funcion para graficar el resultado de la curva ROC

In [ ]:
def curve_roc(probability, title, value, target_test):
    """
    La funcion grafica los resultados de las probabilidades usando la curva de ROC.
    Esta funcion se utiliza en cada modelo despues de realizadas las predicciones.
    
    Parametros:
    probability: son las probabilidades que tiene la clase 1 de ser predichas por el modelo.
    title:  es el mombre del modelo usado.
    value: es el valor de la metrica obtenida.
    
    Return
    La funcion genera el grafico de la curva de ROC
    """
    # Curva ROC
    target_test = target_test.astype(int)
    title = 'Curva ROC - ' + title
    fpr, tpr, thresholds = roc_curve(target_test, probability)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label=f"AUC-ROC = {value:.2f}")
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label="Random Guess")
    plt.xlabel("Tasa de Falsos Positivos (FPR)")
    plt.ylabel("Tasa de Verdaderos Positivos (TPR)")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

Funcion para calcular las metricas objetvo del proyecto

In [ ]:
def calculate_metrics(model, predictions, resampled, feature_test, target_test, time, name_model):
        """
        La funcion calcula las metricas del modelo que se solicitan en el proyecto para poder validar la exactitud de las
        predicciones realizadas y guarda los datos en la lista correspondiente. Estos datos se almacena en listas diferentes para 
        poder identificar los resultados de trabajar con datos sin y con sobremuestreo
    
        Parametros:
        model: es el modelo que se entreno.
        predictions: son las predicciones realizadas por el modelo entrenado.
        feature_test: son las caracteristicas que se usan para calcular las probabilidades de que el modelo prediga la clase positiva.
        target_test: son los objetivos que se usan para hallar las metricas accuracy y la curva de roc.
        time: es el tiempo que tardo el modelo en entrenarse y generar predicciones.
        name_model: es el nombre del modelo que permite el numero del modelo y el algoritmo utilizado.
    
        Retorno:
        La funcion no tiene retorno. Los datos obtenidos se muestran en pantalla y se almacenan en listas para ser procesados posteriormente.
        """
        # Se obtienen las probabilidades que tiene el modelo para predecir la clase positiva
        probabilities = model.predict_proba(feature_test)[:, 1]  # Probabilidades para la clase positiva
        
        # Cálculo de métricas
        accuracy = accuracy_score(target_test, predictions)
        auc_roc = roc_auc_score(target_test, probabilities)  
        logloss = log_loss(target_test, probabilities)

        print(f"Exactitud (Accuracy): {accuracy:.2f}")
        print(f"AUC-ROC: {auc_roc:.2f}")
        print(f"Valor logLoss : {logloss:.2f}")
        
        # Se obtiene la clasificacion del valor obtenido en la metrica de la curva ROC
        classification_metric = result_classification(auc_roc)
        print('Clasificion SP obtenida por el modelo: ', classification_metric)
         
        # Se genera el grafico de la curva de ROC
        curve_roc(probabilities, name_model, auc_roc, target_test)
        
        # Se obtienen los mejores valores de hiperparametros
        parameters = grid_search.best_params_
    
        if resampled == 'No':
            # Se guardan los resultados entregados por el modelo sin sobremuestreo
            models_results.append({
            'Modelo': name_model,
            'Parametros':parameters,
            'Segundos':round(time,2),
            'SP':classification_metric,
            'AUC-ROC': round(auc_roc,2),
            'Precision': round(accuracy,2),
            'log_loss':round(logloss,2),    
            'Sobremuestreo': resampled
             })
        else:
            # Se guardan los resultados entregados por el modelo
            models_results_resampled.append({
            'Modelo': name_model,
            'Parametros':parameters,
            'Segundos':round(time,2),
            'SP':classification_metric,
            'AUC-ROC': round(auc_roc,2),
            'Precision': round(accuracy,2),
            'log_loss':round(logloss,2),
            'Sobremuestreo': resampled
             })

In [ ]:
# Lista para guardar las metricas generadas por los modelos sin hacer ajustes en el balance de clases
models_results = []

# Lista para guardar las metricas generadas por los modelos con sobremuestreo en el balance de clases
models_results_resampled = []

Funcion para entrenar el modelo y generar predicciones

In [ ]:
def train_model_generate_predictions(model, parameters, f_train, t_train, f_test):
    """
    La funcion recibe el modelo predictivo, los parametros para el modelo y los conjuntos de datos para entrenarse
    y generar predcciones.

    Parametros:
    model: es el modelo que entrenara y generara las predicciones.
    parameters: es el conjunto de los mejores que usara el modelo para su entrenamiento.
    f_train: son las caracteristicas que usara el modelo para su entrenamiento.
    t_train: es el conjunto de datos que se usaran como objetivo en el entrenamiento.

    Return
    La funcion retorna los siguientes datos:
    grid_search: es el modelo que se usara para poder generar las predicciones de la clase positiva y asi encontrar el valor de
    la curva de ROC.
    predictions: son los resultados entregados por el modelo despues de ser entrenado que se usan para hallar la metrica accuracy.
    total_time: es el tiempo en minutos que tarda el modelo en ser entrenado y generar predicciones. Este dato se almacena en una lista
    para despues poder comparar los resultados de todos los modelos entrenados.
    """
    # Se inicializan las variables que llevan el control del tiempo que tarda el modelo en ser entrenado y generar predicciones
    start_time = 0
    end_time = 0
    total_time = 0
    
    # Configuracion de la validación cruzada con GridSearchCV
    grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    cv=2,                # Número de particiones de validación cruzada
    scoring='accuracy',  # Métrica de evaluación
    n_jobs=-1            # Usar todos los núcleos disponibles
    )

    # Inicia el tiempo de ejecucion del modelo
    start_time = time.time()
    
    # Entrenar con los datos de entrenamiento sin sobremuestreo
    grid_search.fit(f_train, t_train)
    
    # Evaluar el modelo con los mejores parámetros en los datos de prueba
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(f_test)
    
    # Finaliza el tiempo de ejecucion del modelo
    end_time = time.time()
    
    # Calcula los segudos que tarda el modelo en su entrenamiento y en hacer las predicciones
    total_time = end_time - start_time

    return grid_search, predictions, total_time

Defincion de hiperparametros para ***LogisticRegression***

In [ ]:
# Se eliminan posibles espacios en blanco en los nombres de las columnas
features_train.columns = features_train.columns.str.replace(" ", "_")
features_train_resampled.columns = features_train_resampled.columns.str.replace(" ", "_")

Entrenamiento y validacion de los modelos predictivos.

In [ ]:
# Rejilla de hiperparametros
param_grid_LogisticRegression = {
    "penalty": ["l2"],             # Regularización aplicada (l2 es Ridge regularization)
    "solver": ["lbfgs"],           # Algoritmo de optimización utilizado para ajustar el modelo
    "C": [0.01, 0.1, 1, 10, 100],  # Inverso de la fuerza de regularización; valores más altos implican menos regularización
    "max_iter": [100, 200, 1000]    # Número máximo de iteraciones permitidas para la convergencia del solver
}

# Creacion del modelo
model_1_logistic_regression = LogisticRegression(random_state=42)

In [ ]:
# Se crea un diccionario para guardar los nombres y los modelos con sus respectivos parametros
models = {}

# Se crea un diccionario para guardar los nombres y los modelos con sus respectivos parametros de los modelos con sobremuestreo
models_resampled = {}

***Modelo LogisticRegression***

Datos sin sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_1_logistic_regression, param_grid_LogisticRegression, features_train, target_train, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'No', features_test, target_test, total_time, 'Model_1_LogisticRegression')

***Modelo LogisticRegression***

Datos con sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_1_logistic_regression, param_grid_LogisticRegression, features_train_resampled, target_train_resampled, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'Si', features_test, target_test, total_time, 'Model_1_LogisticRegression_resampled')

In [ ]:
# Se guarda el nombre y la configuracion del modelo para usar con SHAP
models['Model_1_LogisticRegression'] = grid_search.best_estimator_

models_resampled['Model_1_1_LogisticRegression'] = grid_search.best_estimator_

Defincion de hiperparametros para ***RandomForestClassifier***

In [ ]:
# Rejilla de hiperparametros
param_grid_RandomForestClassifier = {
    'n_estimators': [10, 20, 40],         # Numero de arboles en el bosque
    'max_depth': [5, 10, 15],             # Profundidad maxima de los arboles
    'max_features': ['sqrt', 'log2'],     # Numero maximo de caracteristcas a considerar en cada division
    'min_samples_split': [2, 5, 10]       # Numero minimo de muestras necesarias para dividir un nodo interno
}

# Creacion del modelo
model_2_Random_Forest_Classifier = RandomForestClassifier(random_state=42)

***Modelo RandomForestClassifier***

Datos sin sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_2_Random_Forest_Classifier, param_grid_RandomForestClassifier, features_train, target_train, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'No', features_test, target_test, total_time, 'Model_2_RandomForestClassifier')

***Modelo RandomForestClassifier***

Datos con sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_2_Random_Forest_Classifier, param_grid_RandomForestClassifier, features_train_resampled, target_train_resampled, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'Si', features_test, target_test, total_time, 'Model_2_RandomForestClassifier_resampled')

In [ ]:
# Se guarda el nombre y la configuracion del modelo
models['Model_2_RandomForestClassifier'] = grid_search.best_estimator_
models_resampled['Model_2_2_RandomForestClassifier'] = grid_search.best_estimator_

Defincion de hiperparametros para ***XGBClassifier***

In [ ]:
# Rejilla de hiperparametros
param_grid_XGBClassifier = {
    'n_estimators': [10, 20, 40],              # Numero de arboles en el bosque
    'max_depth': [5, 10, 15],                  # Profundidad maxima de los arboles
    'colsample_bytree': [0.5, 0.7, 1.0],       # Proporcion de caracteristicas usadas por arbol
    'min_child_weight': [1, 3, 5]              # Peso minimo total requerido en un nodo hoja
}

# Creacion del modelo
model_3_XGBClassifier = XGBClassifier(random_state=42, eval_metric='logloss')

***Modelo XGBClassifier***

Datos sin sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_3_XGBClassifier, param_grid_XGBClassifier, features_train, target_train, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'No', features_test, target_test, total_time, 'Model_3_XGBClassifier')

***Modelo XGBClassifier***

Datos con sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_3_XGBClassifier, param_grid_XGBClassifier, features_train_resampled, target_train_resampled, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'Si', features_test, target_test, total_time, 'Model_3_XGBClassifier_resampled')

In [ ]:
# Se guarda el nombre y la configuracion del modelo
models['Model_3_XGBClassifier'] = grid_search.best_estimator_
models_resampled['Model_3_3_XGBClassifier'] = grid_search.best_estimator_

Defincion de hiperparametros para ***LGBMClassifier***

In [ ]:
# Rejilla de hiperparametros
param_grid_LGBMClassifier = {
    'n_estimators': [10, 20, 40],               # Numero de arboles en el bosque
    'max_depth': [5, 10, 15],                   # Profundidad maxima de los arboles
    'colsample_bytree': [0.5, 0.7, 1.0],        # Proporcion de caracteristicas usadas por arbol
    'min_child_samples': [1, 3, 5]              # Numero minimo requerido de muestras en una hoja
}

# Creacion del modelo
model_4_LGBMClassifier = LGBMClassifier(random_state=42, class_weight='balanced')

***Modelo LGBMClassifier***

Datos sin sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_4_LGBMClassifier, param_grid_LGBMClassifier, features_train, target_train, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'No', features_test, target_test, total_time, 'Model_4_LGBMClassifier')

***Modelo LGBMClassifier***

Datos con sobre muestreo

In [ ]:
# LLamar a la funcion para entregar el modelo y generar predicciones
grid_search, predictions, total_time = train_model_generate_predictions(model_4_LGBMClassifier, param_grid_LGBMClassifier, features_train_resampled, target_train_resampled, features_test)

# LLamar a la funcion para calcular las metricas objetivo
calculate_metrics(grid_search, predictions, 'Si', features_test, target_test, total_time, 'Model_4_LGBMClassifier_resampled')

In [ ]:
# Se guarda el nombre y la configuracion del modelo
models['Model_4_LGBMClassifier'] = grid_search.best_estimator_
models_resampled['Model_4_4_LGBMClassifier'] = grid_search.best_estimator_

## 5. Interpretación de Resultados

### Importancia de características:
- Identificar las variables que más contribuyen a la predicción

Analisis de caracteristicas sin sobre muestreo y definicion del umbral para eliminar las caracteristicas de menos relevancia para las predicciones.

In [ ]:
dfs_features_train_shap = {}

for name, model in models.items():
    print(f"Analisis SHAP para {name} \n")
    
    if name == "Model_2_RandomForestClassifier":
        explainer = shap.Explainer(model, features_train)  
        shap_values = explainer(features_test, check_additivity=False)
        mean_shap = shap_values.values.mean(axis=0)
        
        # Reducir dimensiones para que mean_shap sea un vector
        if mean_shap.ndim == 2:
        # Tomar promedio absoluto a través de las clases
           mean_shap = np.abs(mean_shap).mean(axis=1)
    
    else:
        # Para otros modelos como logisticregression, xgbclassifier, lgbmclassifier
        explainer = shap.Explainer(model, features_train)  
        shap_values = explainer(features_test)
        mean_shap = shap_values.values.mean(axis=0)
        
    # Procesar los valores SHAP como antes
    print('shape mean_shap', mean_shap.shape)
    print('Validacion de longitures para features_train y mean_shap -->', len(features_train.columns), len(mean_shap))
    
    features_importance = pd.DataFrame({'feature': features_train.columns, 'Mean SHAP Value': mean_shap})
    features_importance = features_importance.sort_values(by='Mean SHAP Value', ascending=False)
    
    print(f'Caracteristicas mas importantes para {name} :\n')
    display(features_importance.head(20))

    # Definicion del umbral del 5% del valor maximo
    threshold = 0.05 * features_importance['Mean SHAP Value'].max()
    features_to_remove = features_importance[features_importance['Mean SHAP Value'] < threshold]['feature'].values
    features_train_shap = features_train.drop(columns=features_to_remove)
    dfs_features_train_shap[name] = features_train_shap
    view_columns = pd.DataFrame(dfs_features_train_shap[name])

    print('')
    print('Numero de caracteristicas antes de apilcar shap -->: ', features_train.shape[1])
    print('Numero de caracteristicas despues de apilcar shap -->: ', view_columns.shape[1])
    
    print('-' * 120)
    print('')

Analisis de caracteristicas con sobre muestreo y definicion del umbral para eliminar las caracteristicas de menos relevancia para las predicciones

In [ ]:
# Diccionario para almacenar las columnas que superan el umbral definido despues de hallar los valores SHAP
dfs_features_train_resampled_shap = {}

for name, model in models_resampled.items():
    print(f"Analisis SHAP para {name} \n")
    
    if name == "Model_2_2_RandomForestClassifier":
        explainer = shap.Explainer(model, features_train_resampled)  # Eliminamos el 'masker'
        shap_values = explainer(features_test, check_additivity=False)
        mean_shap = shap_values.values.mean(axis=0)
    
       
        # Reducir dimensiones para que mean_shap sea un vector
        if mean_shap.ndim == 2:
        # Tomar promedio absoluto a través de las clases
           mean_shap = np.abs(mean_shap).mean(axis=1)    
        
    else:
        # Para otros modelos como logisticregression, xgbclassifier, lgbmclassifier
        explainer = shap.Explainer(model, features_train_resampled)  # Eliminamos el 'masker'
        shap_values = explainer(features_test)
        mean_shap = shap_values.values.mean(axis=0)


    
    # Procesar los valores SHAP como antes
    print('shape mean_shap', mean_shap.shape)
    print('Validacion de longitures para features_train y mean_shap -->', len(features_train_resampled.columns), len(mean_shap))
    
    features_resampled_importance = pd.DataFrame({'feature': features_train_resampled.columns, 'Mean SHAP Value': mean_shap})
    features_resampled_importance = features_resampled_importance.sort_values(by='Mean SHAP Value', ascending=False)
    
    print(f'Caracteristicas mas importantes para {name} :\n')
    display(features_resampled_importance.head(20))

    # Definicion del umbral del 5%
    threshold = 0.05 * features_resampled_importance['Mean SHAP Value'].max()
    features_to_remove = features_resampled_importance[features_resampled_importance['Mean SHAP Value'] < threshold]['feature'].values
    features_train_resampled_shap = features_train_resampled.drop(columns=features_to_remove)
    dfs_features_train_resampled_shap[name] = features_train_resampled_shap
    view_columns = pd.DataFrame(dfs_features_train_resampled_shap[name])

    print('')
    print('Numero de caracteristicas antes de apilcar shap -->: ', features_train_resampled.shape[1])
    print('Numero de caracteristicas despues de apilcar shap -->: ', view_columns.shape[1])
    
    print('-' * 120)
    print('')

### Resultados esperados:
- Lograr al menos un AUC-ROC de 0.81 para cumplir con los criterios de evaluación de 4.5 SP.



***Modelo LogisticRegression usando SHAP***

In [ ]:
# Creacion del modelo
model_1_logistic_regression = LogisticRegression(random_state=42)

Datos sin sobre muestreo

In [ ]:
features_shap = pd.DataFrame(dfs_features_train_shap['Model_1_LogisticRegression'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_shap, target_train, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_1_logistic_regression, param_grid_LogisticRegression, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'No', features_test_shap, target_test_shap, total_time, 'Model_1_LogisticRegression_Shap')

Datos con sobre muestreo

In [ ]:
features_resampled_shap = pd.DataFrame(dfs_features_train_resampled_shap['Model_1_1_LogisticRegression'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_resampled_shap, target_train_resampled, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_1_logistic_regression, param_grid_LogisticRegression, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'Si', features_test_shap, target_test_shap, total_time, 'Model_1_LogisticRegression_resampled_Shap')

***Modelo RandomForestClassifier usando SHAP***

In [ ]:
# Creacion del modelo
model_2_Random_Forest_Classifier = RandomForestClassifier(random_state=42)

Datos sin sobre muestreo

In [ ]:
features_shap = pd.DataFrame(dfs_features_train_shap['Model_2_RandomForestClassifier'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_shap, target_train, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_2_Random_Forest_Classifier, param_grid_RandomForestClassifier, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'No', features_test_shap, target_test_shap, total_time, 'Model_2_RandomForestClassifier_shap')

Datos con sobre muestreo

In [ ]:
features_resampled_shap = pd.DataFrame(dfs_features_train_resampled_shap['Model_2_2_RandomForestClassifier'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_resampled_shap, target_train_resampled, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_2_Random_Forest_Classifier, param_grid_RandomForestClassifier, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'Si', features_test_shap, target_test_shap, total_time, 'Model_2_RandomForestClassifier_resampled_Shap')

***Modelo XGBClassifier usando SHAP***

In [ ]:
# Creacion del modelo
model_3_XGBClassifier = XGBClassifier(random_state=42, eval_metric='logloss')

Datos sin sobre muestreo

In [ ]:
features_shap = pd.DataFrame(dfs_features_train_shap['Model_3_XGBClassifier'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_shap, target_train, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_3_XGBClassifier, param_grid_XGBClassifier, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'No', features_test_shap, target_test_shap, total_time, 'model_3_XGBClassifier_shap')

Datos con sobre muestreo

In [ ]:
features_resampled_shap = pd.DataFrame(dfs_features_train_resampled_shap['Model_3_3_XGBClassifier'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_resampled_shap, target_train_resampled, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_3_XGBClassifier, param_grid_XGBClassifier, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'Si', features_test_shap, target_test_shap, total_time, 'model_3_XGBClassifier_resampled_Shap')

***Modelo LGBMClassifier usando SHAP***

In [ ]:
# Creacion del modelo
model_4_LGBMClassifier = LGBMClassifier(random_state=42)

Datos sin sobre muestreo

In [ ]:
features_shap = pd.DataFrame(dfs_features_train_shap['Model_4_LGBMClassifier'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_shap, target_train, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_4_LGBMClassifier, param_grid_LGBMClassifier, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'No', features_test_shap, target_test_shap, total_time, 'Model_4_LGBMClassifier_shap')

Datos con sobre muestreo

In [ ]:
features_resampled_shap = pd.DataFrame(dfs_features_train_resampled_shap['Model_4_4_LGBMClassifier'])

features_train_shap, features_test_shap, target_train_shap, target_test_shap = train_test_split(features_resampled_shap, target_train_resampled, random_state=12345, test_size=0.2)

In [ ]:
grid_search, predictions, total_time = train_model_generate_predictions(model_4_LGBMClassifier, param_grid_LGBMClassifier, features_train_shap, target_train_shap, features_test_shap)

calculate_metrics(grid_search, predictions, 'Si', features_test_shap, target_test_shap, total_time, 'Model_4_LGBMClassifier_resampled_Shap')

Se listan los resultados entregados por los modelos entrenos con y sin sobremuestreo y con el uso de SHAP

In [ ]:
# Ajusta la configuración para mostrar más caracteres
pd.set_option('display.max_colwidth', None)

# Se muestran en pantalla los resultados obtenidos por cada modelo en los diferentes escenarios generados
df_models_results = pd.DataFrame(models_results)
df_models_results_resampled = pd.DataFrame(models_results_resampled)
display(df_models_results)
display(df_models_results_resampled)

In [ ]:
# Se muestran los resultados de los dos mejores modelos
best_exactitud = 0
best_model = ''
best_auc = 0

print('Datos del mejor modelo sin sobremuestreo')
for index, row in df_models_results.iterrows():
   if row['AUC-ROC'] >= best_auc:
      best_auc = row['AUC-ROC']
      if row['Precision'] >= best_exactitud:
         best_exactitud = row['Precision']
         best_model = row

print(best_model)
print('')            


best_auc = 0
best_exactitud = 0
best_model = ''

print('')
print('Datos del mejor modelo con sobremuestreo')
for index, row in df_models_results_resampled.iterrows():
    if row['AUC-ROC'] >= best_auc:
       best_auc = row['AUC-ROC']
       if row['Precision'] >= best_exactitud:
          best_exactitud = row['Precision']
          best_model = row
                    
print(best_model)

***Analisis mejores modelos***

Para el entrenamiento de los modelos predictivos se aplico sobre muestreo y SHAP para poder evaluar los cambios entregados en cada uno de ellos al medir las metricas objetivo.

#### Datos sin sobre muestreo

- El modelo que mejor desempeño presento fue el basado en ***XGBClassifier***, alcanzando el valor maximo el la metrica SP.

#### Datos con sobre muestreo

- El modelo que mejor desempeño presento fue el basado en ***RandomForestClassifier***, alcanzando el valor maximo el la metrica SP.

Al revisar los datos entregados por cada uno de los modelos mencionados anteriormente, se sugiere trabajar con RandomForestClassifier y datos son sobre muestreo, ya que mejora las metricas de XGBClassifier y toma el mismo tiempo.


Grafico de los resultados entregados por los modelos

In [ ]:
# Definir las columnas numéricas para escalarlas
numeric_columns = ['Segundos', 'SP', 'AUC-ROC', 'Precision']

# Escalar las métricas
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(
    scaler.fit_transform(df_models_results[numeric_columns]),
    columns=numeric_columns,
    index=df_models_results['Modelo']
)

df_resampled_scaled = pd.DataFrame(
    scaler.fit_transform(df_models_results_resampled[numeric_columns]),
    columns=numeric_columns,
    index=df_models_results_resampled['Modelo']
)

# Crear gráfico de barras agrupadas
x = np.arange(len(df_scaled.index))  # Posiciones en el eje x
width = 0.35  # Ancho de las barras

plt.figure(figsize=(16, 6))

# Iterar sobre cada métrica para crear las barras
for i, column in enumerate(numeric_columns):
    plt.bar(
        x - width/2 + i * width / len(numeric_columns),  # Posición ajustada de las barras
        df_scaled[column],
        width=width / len(numeric_columns),
        label=f'{column} (original)',
        alpha=0.7
    )
    plt.bar(
        x + width/2 + i * width / len(numeric_columns),  # Posición ajustada de las barras
        df_resampled_scaled[column],
        width=width / len(numeric_columns),
        label=f'{column} (resampled)',
        alpha=0.7,
        hatch='/'  # Añadir patrón a las barras para diferenciarlas
    )

# Etiquetas y leyenda
plt.title('Comparación de resultados con y sin sobremuestreo en los datos', fontsize=14)
plt.xlabel('Modelos', fontsize=12)
plt.ylabel('Métricas Escaladas', fontsize=12)
plt.xticks(ticks=x, labels=df_scaled.index, rotation=45)  # Usar nombres de modelos en el eje x

# Colocar la leyenda fuera del gráfico a la derecha
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), ncol=1, fontsize=10)  # Ajuste fuera del gráfico
plt.grid(axis='y', linestyle='--', alpha=0.6)  # Líneas de guía solo en el eje y
plt.tight_layout(pad=3.0)  # Aumentar el espacio alrededor de los elementos

plt.show()

El grafico anterior corrobora los resultados entregados por el modelo ***RandomForestClassifier*** usando datos con sobre muestreo. Se puede observar el poco tiempo que toma su entrenamuneto, generacion de resulatdos y un mayor valor en la clasifiacion SP.

In [ ]:
# Contar frecuencias de nombres en ambos diccionarios
counter_resampled = Counter()
counter_original = Counter()

for key, names in dfs_features_train_resampled_shap.items():
    counter_resampled.update(names)

for key, names in dfs_features_train_shap.items():
    counter_original.update(names)

# Combinar los contadores en un DataFrame
all_names = set(counter_resampled.keys()).union(set(counter_original.keys()))
data = {
    "Name": list(all_names),
    "Resampled": [counter_resampled[name] for name in all_names],
    "Original": [counter_original[name] for name in all_names],
}
df = pd.DataFrame(data)

# Ordenar descendentemente por la suma de las frecuencias
df["Total"] = df["Resampled"] + df["Original"]
df = df.sort_values(by="Total", ascending=False).drop(columns=["Total"])

# Graficar las frecuencias
df.set_index("Name").plot(kind="bar", figsize=(10, 6))
plt.title("Frecuencia de las columnas que superan el umbral aplicado al valor maximo dado por SHAP")
plt.ylabel("Frecuencia")
plt.xlabel("Nombre columna")
plt.xticks(rotation=90)
plt.legend(title="Fuente")
plt.tight_layout()
plt.show()

In [ ]:
# Contar la frecuencia de todas las columnas
all_columns = [col for columns in dfs_features_train_shap.values() for col in columns]
column_counter = Counter(all_columns)

# Ordenar de mayor a menor por frecuencia
sorted_columns = column_counter.most_common()
columns = [item[0] for item in sorted_columns]  # Nombres de columnas
frequencies = [item[1] for item in sorted_columns]  # Frecuencias

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(columns, frequencies, color='skyblue', edgecolor='black')
plt.xlabel('Nombres de Columnas', fontsize=6)
plt.ylabel('Frecuencia')
plt.title('Frecuencia de columnas en los datos sin sobre muestreo')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Contar la frecuencia de todas las columnas
all_columns = [col for columns in dfs_features_train_resampled_shap.values() for col in columns]
column_counter = Counter(all_columns)

# Ordenar de mayor a menor por frecuencia
sorted_columns_resampled = column_counter.most_common()
columns = [item[0] for item in sorted_columns]  # Nombres de columnas
frequencies = [item[1] for item in sorted_columns]  # Frecuencias

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(columns, frequencies, color='green', edgecolor='black')
plt.xlabel('Nombres de Columnas', fontsize=6)
plt.ylabel('Frecuencia')
plt.title('Frecuencia de columnas en los datos con sobre muestreo')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
print('Columnas que mas se repiten con SHAP sin sobremuestreo')

for col, freq in counter_original.items():  # Usar .items() para iterar por pares clave-valor
    if freq > 1:
        print(f"Columna: {col}, Frecuencia: {freq}")

print('')
print('Columnas que mas se repiten con SHAP con sobremuestreo')
for col, freq in counter_resampled.items():  # Usar .items() para iterar por pares clave-valor
    if freq > 1:
        print(f"Columna: {col}, Frecuencia: {freq}")

# Plan de Trabajo

## 1. Definición del Problema

### Objetivo: 

Predecir la probabilidad de cancelación que tienen los clientes.

#### Métricas objetivo:

##### - ***Métrica principal***: AUC-ROC (Área bajo la curva de la curva ROC).
##### - ***Métricas adicionales***:
    - Exactitud.
    - log_loss.
    
#### Criterios de evaluación:

- AUC-ROC < 0.75: 0 SP.
- 0.75 ≤ AUC-ROC < 0.81: 4 SP.
- 0.81 ≤ AUC-ROC < 0.85: 4.5 SP.
- 0.85 ≤ AUC-ROC < 0.87: 5 SP.
- 0.87 ≤ AUC-ROC < 0.88: 5.5 SP.
- AUC-ROC ≥ 0.88: 6 SP.

### 1.1 Carga de los datos

- Carga de los diferentes conjuntos de datos en formato dataframe.
  
## 2. Entendimiento del Conjunto de Datos

### Revisión inicial:
- Analizar la columna EndDate para confirmar que los clientes con valor ***"No"*** están etiquetados como no cancelados.
- Identificar el balance de clases (proporción de cancelados vs. no cancelados).
- Revisar los tipos de datos acorde a la informacion que representan cada una de las características disponibles.
- Analizar los rangos de valores en cada columna.
- Analizar relevancia de las diferentes columnas en los resultados de las predicciones.

***Analisis datos columna*** EndDate

- ***Valores unicos***: los valores unicos no presentan inconsistencias o rangos no permitidos.
- ***Balance de clases***:

    - ***73.46%*** de los clientes tienen su contrato vigente- 
    - ***26.54%*** de los clientes han cancelado.
    - ***Nota***: esto siguiere la posibilidad de usar sobremuestreo para balancear las clases.
      
#### Preguntas clave:

- ***¿Qué variables parecen correlacionarse más con la cancelación?***

    - Se identificaron como relevantes las siguientes columnas:
 
        - ***df_personal***: gender, seniorcitizen
            
        - ***df_contract***: paymentmethod, totalcharges
            
        - ***df_internet***: techsupport
            
        - ***df_phone***: multipleLines
            
- ***¿Hay variables redundantes o irrelevantes que se puedan eliminar?***

    - No se observan columnas con datos irrelevantes, sin embargo se deben hacer pruebas de incidencia en los resultados para poder determinar la relevancia de caracteristicas y asi depurar los datos para una mejor interpretacion por parte del modelo.

## 3. Preparación de los Datos

### 3.1 Limpieza de datos:

- Pasar a minuscula los nombres de las columnas.
- ***Codificacion binaria***: Tranformar columnas como partner, dependents paperlessbilling y multiplelines en valores 1 y 0.
- Buscar registros duplicados.
- Identificar y procesar valores ausentes.
    ##### ***Hallazgos***
  
        - df_contratc

            - Columna enddate: ***5174 valores ausentes*** (73.46%). Estos valores corresponden a contratos vigentes, por lo que no es necesario imputarlos.

            - columna totalcharges: ***11 valores ausentes*** (0.16%) que representan el ***0.16%*** de los datos. Estos registros seran eliminados debido a su baja representatividad.
  
- Homogeneizar texto en columnas tipo object (minusculas, sin espacios en blanco).
- Revisar y ajustar valores minimos, maximos y unicos en cada columna.
- Modificar los tipos de datos segun la informacion representada.

    ##### ***Analisis valores minimos, maximos y unicos***

Al revisar los valores minimo y maximo de todos los conjuntos de datos, se pueden observar las siguientes novedades:

- df_personal:
  
    - La columna seniorcitizen maneja valores de 0 y 1 con un tipo de dato object. Se pasara int para disminuir el consumo de memoria al almecenar el dato.
      
- df_contract:
  
    - La columna begindate maneja fechas con un tipo de dato object. Se pasara a tipo datetime para procesar correctamente los datos de fecha.
    - La columna enddate maneja fechas con un tipo de dato object. Se pasara a tipo datetime para procesar correctamente los datos de fecha.
    - La columna totalcharges maneja datos numericos con un tipo de dato objetc. Se pasara a float64 para procesar correctamente los datos.

Las demas columnas tiene un tipo de dato acorde a la informacion que representan.


Los ***valores unicos*** encontrados en las diferentes columnas estan dentro de los rangos permitidos segun la informacion de representan, sin embargo, la columna begindate del conjunto de datos df_contract representa fechas, estos datos estan estructurados como object; la columna de pasara a datetime para que las fechas se procesen adecuadamente.
    
- 3.2 Codificación y transformación:
 
    - Consolidar los datos en un único registro por cliente.
        - Revisar valores ausentes en el nuevo conjunto de datos.
        - Crear la columna objetivo basada en enddate.
    - Codificar variables categóricas, como métodos de pago y tipos de contrato, utilizando técnicas como One-Hot Encoding.
    - Escalar variables numéricas.
      
- 3.3 Análisis exploratorio de datos (EDA):
  
    - Visualizar la distribución de clases (cancelados vs. no cancelados).

## 4. Construcción del Modelo Predictivo

### 4.1 División de datos:

- Dividir los datos en entrenamiento (80%) y prueba (20%), asegurando una proporción equilibrada entre clases.
  
### 4.2 Selección del modelo:

- Evaluar los siguientes algoritmos_
- Regresión logística.
- Random Forest Classifier.
- Gradient Boosting (XGBoost, LightGBM).
- Comparar los modelos utilizando AUC-ROC como métrica principal.
- 
### 4.3 Evaluación del modelo:

- Validación cruzada para asegurar la estabilidad.
- Comparar modelos basandose en AUC-ROC,  exactitud y log_loss.
  
### 4.4 Optimización del modelo:

- Ajustar hiperparámetros utilizando técnicas como Grid Search o Random Search para maximizar AUC-ROC.
- Equilibrar clases mediante técnicas como sobremuestreo (SMOTE) o submuestreo si hay desbalance.

## 5. Interpretación de Resultados

### Importancia de características:

- Identificar variables clave con la herramienta SHAP.

        Se define un ***umbral del 5%*** del valor maximo obtenido por SHAP al aplicarlo a los modelos, obteniendo los siguiente resultados (5 columnas mas representativas):

        - Regresión logística sin sobremuestreo                   Regresión logística con sobremuestreo

        - type_month-to-month.                                    - monthlycharges.          
        - monthlycharges.                                         - totalcharges.
        - totalcharges.                                           - deviceprotection_1.
        - mesbegindate_3.                                         - mesbegindate_4.
        - diabegindate_3.                                         - diabegindate_4.
  
  
        - Random Forest Classifier sin sobremuestreo              Random Forest Classifier con sobremuestreo

        - type_month-to-month.                                    - type_month-to-month.
        - totalcharges.                                           - yearbegindate.
        - internetservice_fiber_optic.                            - totalcharges.
        - yearbegindate.                                          - paymentmethod_electronic_check.
        - type_one_year.                                          - type_two_year.
  
  
        - Gradient Boosting XGBoost sin sobremuestreo             Gradient Boosting XGBoost con sobremuestreo

        - totalcharges.                                           - mesbegindate_2.
        - type_month-to-month.                                    - type_one_year.
        - mesbegindate_1.                                         - mesbegindate_10.
        - streamingmovies_0.                                      - mesbegindate_12.
        - monthlycharges.                                         - paymentmethod_bank_transfer_(automatic).
  
  
        - Gradient Boosting LightGBM sin sobremuestreo            Gradient Boosting LightGBM sin sobremuestreo

        - totalcharges.                                           - mesbegindate_12.
        - type_month-to-month.                                    - mesbegindate_2.
        - streamingmovies_0.                                      - mesbegindate_10.
        - onlinebackup_0.                                         - paymentmethod_bank_transfer_(automatic).
        - dependents_0.                                           - paperlessbilling_1.

En el analisis de las columnas en los diferentes modelos entrenados con datos sin y con sobremuestreo, en primer lugar se observa la presencia de la columna ***totalcharges*** y en segundo lugar esta la columna ***type_month-to-month***. Las demas columnas no tienen un patron tan marcado como las dos mencionadas anteriormente. 
Con la presencia de las columnas ***totalcharges*** y ***type_month-to-month*** como las mas representativas en las predicciones de las posibilidades de que el cliente cancele el contrato, se pueden realizar estudios de comportamiento con respecto al cargo total que paga el cliente y el tipo de contrato que tiene, para con base en los resultados, poder implementar estrategias comerciales que lleven a una mayor permanencia de los clientes con sus respectivos contratos.

### Resultados esperados:
.
- AUC-ROC >= 0.81 para para obtener al menos 4.5 SP.

Acontinuacion se listan los diferentes resultados entregados por cada modelo:

***Resultados sin sobremuestreo***

| Modelo                      | Segundos | SP   | AUC-ROC | Precision | log_loss | Sobremuestreo |
|-----------------------------|----------|------|---------|-----------|----------|---------------|
| LogisticRegression          | 67.49    | 4.5  | 0.85    | 0.80      | 0.42     | No            |
| RandomForestClassifier      | 14.66    | 5.0  | 0.87    | 0.82      | 0.40     | No            |
| XGBClassifier               | 22.56    | 6.0  | 0.91    | 0.86      | 0.32     | No            |
| LGBMClassifier              | 73.04    | 6.0  | 0.90    | 0.81      | 0.38     | No            |
| LogisticRegression_Shap     | 2.23     | 4.5  | 0.82    | 0.80      | 0.45     | No            |
| RandomForestClassifier_shap | 10.87    | 5.5  | 0.87    | 0.83      | 0.38     | No            |
| XGBClassifier_shap          | 5.46     | 4.0  | 0.80    | 0.78      | 0.45     | No            |
| LGBMClassifier_shap         | 9.24     | 4.0  | 0.78    | 0.77      | 0.47     | No            |

***Resultados con sobremuestreo***

| Modelo                               | Segundos | SP   | AUC-ROC | Precision | log_loss | Sobremuestreo |
|--------------------------------------|----------|------|---------|-----------|----------|---------------|
| LogisticRegression_resampled         | 2.43     | 4.5  | 0.85    | 0.76      | 0.48     | Sí            |
| RandomForestClassifier_resampled     | 18.80    | 5.0  | 0.86    | 0.82      | 0.45     | Sí            |
| XGBClassifier_resampled              | 56.98    | 6.0  | 0.86    | 0.86      | 0.34     | Sí            |
| LGBMClassifier_resampled             | 31.30    | 6.0  | 0.84    | 0.84      | 0.36     | Sí            |
| LogisticRegression_resampled_Shap    | 0.37     | 4.0  | 0.81    | 0.72      | 0.54     | Sí            |
| RandomForestClassifier_resampled_Shap| 13.35    | 6.0  | 0.94    | 0.86      | 0.34     | Sí            |
| XGBClassifier_resampled_Shap         | 6.69     | 4.0  | 0.68    | 0.68      | 0.54     | Sí            |
| LGBMClassifier_resampled_Shap        | 11.29    | 4.0  | 0.69    | 0.69      | 0.58     | Sí            |

***Comparacion de resultados con y sin sobremuestreo***

| Modelo                                   | Segundos | SP    | AUC-ROC | Precision | log_loss | Sobremuestreo |
|------------------------------------------|----------|-------|---------|-----------|----------|---------------|
| LogisticRegression (original vs resampled)         | 65.06   | 0.0   | 0.00    | 0.04      | -0.06    | No vs Sí     |
| RandomForestClassifier (original vs resampled) | -4.14 | 0.0 | 0.01 | 0.00 | -0.05 | No vs Sí |
| **XGBClassifier (original vs resampled)**              | **-34.42**  | **0.0**   | **0.05**    | **0.00**      | **-0.02**    | **No vs Sí**     |
| LGBMClassifier (original vs resampled)             | 41.74   | 0.0   | 0.06    | -0.03     | 0.02     | No vs Sí     |
| LogisticRegression_Shap (original vs resampled)    | 1.86    | 0.5   | 0.01    | 0.08      | -0.09    | No vs Sí     |
| **RandomForestClassifier_Shap (original vs resampled)** | **-2.48** | **-0.5** | **-0.07** | **-0.03** | **0.04** | **No vs Sí** |
| XGBClassifier_Shap (original vs resampled)         | -1.23   | 0.0   | 0.12    | 0.10      | -0.09    | No vs Sí     |
| LGBMClassifier_Shap (original vs resampled)        | -2.05   | 0.0   | 0.09    | 0.08      | -0.11    | No vs Sí     |

En la anterior tabla se resaltan los modelos con mejores resultados. 

***Sin sobremuestreo***:
   - El modelo basado en el algoritmo ***XGBClassifier*** entrego los mejores resultados al balancear de mejor forma los valores generados para cada una de las metricas que se usaron como parametro para determinar la calidad de las predicciones. La siguiente es la seleccion de hiperparametros usados en el modelo: ***'colsample_bytree': 1.0, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 40***

***Con sobremuestreo***:
   - El modelo basado en el algoritmo ***RandomForestClassifier*** entrego los mejores resultados al balancear de mejor forma los valores generados para cada una de las metricas que se usaron como parametro para determinar la calidad de las predicciones. En este entrenamiento se realizo ***sobremuestreo*** y aplico ***SHAP*** para seleccionar las columnas con mayor participacion en la generacion de predicciones. La siguiente es la seleccion de hiperparametros usados en el modelo: ***'max_depth': 15, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 40***

## 6. Reporte y Presentación

### 6.1 ***Portada***

<div style="text-align: center;">
    <strong><em>Título del proyecto</em></strong>: Posibilidad de cancelación del contrato por parte del cliente.<br>
    <strong><em>Presentado por</em></strong>: Jose Luis Fernandez R.<br>
    <strong><em>Fecha</em></strong>: Diciembre 13 de 2024.
</div>

### 6.2 Introducción

- Objetivo general del proyecto: Predecir la probabilidad de cancelación que tienen los clientes.
  
- Problema a resolver y su importancia: Se han detectado cancelaciones frecuentes de contratos por parte de los clientes sin una razon aparente, como resultado situacion afecta de manera directa los ingresos de la empresa y dificulta la posibilidad de mejorar los servicios al desconocer sus puntos debiles en la prestacion de sus servicios.
Breve descripción de los datos utilizados: Para la ejecucion de este proyecto se utilizo la informacion de los clientes registrada en la empresa asi como el listado de los diferentes servicios que se ofrecen. 

### 6.3 Metodología

#### 6.3.1 Descripción del proceso

- Definición del problema: Se desconocen las causas por las cuales los clientes realizan la cancelacion de sus contratos.
  
- Preparación y limpieza de datos: La informacion entregada para le realizacion del proyecto se unifico en un solo conjunto de datos para el procesamiento de valores ausentes y determinar como manejarlos, tambien poder suprimir caracteres que no fueran parte de los datos, analizar los rangos y valores unicos dentro de cada una de las columnas y homogeneizar el uso de minusculas.
  
- Exploración de los datos: En este paso se analizaron los balances de las clases, es decir, en que medida en el conjunto de datos aparece con mayor frecuencia en dato que indica la cancelacion del contrato. Posteriormente para pasar a la eleccion y entrenamiento de modelos, se separaron los conjuntos de datos dando el 80% del total para el entrenamiento y el 20% restante para pruebas de validacion.
  
#### 6.3.2 Selección de modelos

- **Métodos y algoritmos considerados:**
  
    - **Modelos supervisados:** Debido a la naturaleza del problema (predecir la probabilidad de cancelación de contratos), se utilizaron modelos supervisados.
      
    - **Modelos de clasificación:** Los principales algoritmos considerados fueron modelos de clasificación binaria, ya que el objetivo es predecir una variable categórica: si un cliente cancelará o no su contrato.
      
    - **Modelos más utilizados en problemas similares:**
      
        - **Regresión logística:** Modelo sencillo y eficiente para estimar probabilidades de eventos binarios.
      
        - **Árboles de decisión y Random Forest:** Estos modelos permiten identificar características no lineales en los datos y son sencillos de interpretar.

        - **XGBoost:** Un modelo basado en gradiente boosting que ha demostrado un excelente desempeño en tareas de clasificación.

        - **LightGBM:** Similar a XGBoost, es una variante más eficiente que maneja bien grandes volúmenes de datos y es capaz de capturar patrones complejos de forma rápida.

- **Técnicas de procesamiento:**

    - **Over-sampling (sobremuestreo):** En el conjunto de datos entregado existe una mayor presencia de clientes que no han cancelado su contrato, lo que lleva a un desbalance en las clases. Por este motivo, se utilizó el método SMOTE para generar registros con datos de cancelación de contrato y analizar los resultados con este nuevo escenario.

    - **Codificación de variables categóricas:** Las columnas de tipo objeto se codificaron utilizando ***OneHotEncoder***.

    - **Escalado de variables:** Las columnas de tipo numérico se escalaron utilizando ***StandardScaler***.
      
    - **Busqueda de hiperparametros:** Para poder seleccionar el mejor modelo y su configuracion se utilizo ***GridSearchCV*** con validacion cruzada, de esta manera de optimizo el uso del conjunto de datos en la generacion de posibles configuraciones para los diferentes modelos.

- **Justificación de la elección de los modelos:**

    - **Características del problema:**

        - ***Predicción binaria:*** El modelo debe ser capaz de clasificar correctamente a los clientes que cancelan su contrato frente a los que no lo hacen. Por lo tanto, los modelos seleccionados se centraron en la clasificación binaria.

        - ***Interpretabilidad:*** Los modelos como regresión logística y árboles de decisión son fácilmente interpretables, lo cual es relevante para entender los factores clave que influyen en la cancelación de contratos. Este es un valor agregado, ya que los resultados no solo deben ser precisos, sino también explicables para que los responsables de la toma de decisiones puedan actuar sobre ellos.

        - ***Manejo de datos desbalanceados:*** Se debe contemplar la posible desproporción entre clientes que cancelan o no su contrato. Los modelos como ***Random Forest***, ***XGBoost*** y ***LightGBM*** pueden manejar el desbalance mediante técnicas como pesos ajustados en las clases o sampling.

    - **Rendimiento esperado:**

        - Modelos como ***XGBoost*** y ***LightGBM*** han mostrado excelente desempeño en problemas similares, por lo que fueron elegidos para evaluar su rendimiento en el caso de la cancelación de contratos.

        - ***Random Forest*** y ***Árboles de Decisión*** fueron tenidos en cuenta para obtener un modelo robusto con buena capacidad de generalización, ya que son menos propensos al sobreajuste en comparación con algunos otros modelos.

    - **Complejidad computacional:**

        - ***XGBoost*** y ***LightGBM*** ofrecen una ventaja en términos de eficiencia computacional. Al manejar grandes volúmenes de datos y entrenarse rápidamente, se priorizan estos modelos para garantizar que los experimentos se lleven a cabo en un tiempo razonable.

        - Modelos como la ***regresión logística*** fueron considerados como una base para comparar la efectividad de los modelos más complejos, ya que, aunque no son tan potentes, ofrecen un desempeño decente en muchos escenarios y tienen un bajo costo computacional.

#### 6.3.3 Criterios de evaluación:

- **Métricas utilizadas:**
  
  El modelo seleccionado debía obtener una clasificación mínima de ***4.5 SP***. Esta es una métrica proporcionada por la empresa que evalúa el nivel de exactitud del modelo basado en su desempeño en la ***Curva AUC-ROC***. Para ser considerado, el modelo debía alcanzar un valor mínimo de 0.81 en la AUC-ROC. Además, se incluyeron métricas complementarias como ***precisión (accuracy)*** y ***log-loss***, con el propósito de ofrecer una visión más completa del rendimiento del modelo.

- **Relevancia de las métricas:**

  - ***SP (Scoring Performance):***  
    Esta valoración, proporcionada por la empresa, se asigna al modelo con base en los siguientes parámetros:  
    
      - **AUC-ROC < 0.75:** 0 SP.  
      - **0.75 ≤ AUC-ROC < 0.81:** 4 SP.  
      - **0.81 ≤ AUC-ROC < 0.85:** 4.5 SP.  
      - **0.85 ≤ AUC-ROC < 0.87:** 5 SP.  
      - **0.87 ≤ AUC-ROC < 0.88:** 5.5 SP.  
      - **AUC-ROC ≥ 0.88:** 6 SP.

  - ***Curva AUC-ROC (Área Bajo la Curva - Receiver Operating Characteristic):***  
    Esta métrica mide la capacidad del modelo para diferenciar entre clases. Es especialmente relevante en este caso porque permite evaluar cuán bien el modelo identifica a los clientes propensos a cancelar su contrato. Un valor cercano a 1 indica un alto desempeño.

  - ***Precisión (accuracy):***  
    Representa el porcentaje de predicciones correctas del modelo en relación con el total de datos evaluados. Aunque es una métrica útil, debe interpretarse junto con otras métricas, especialmente en problemas con clases desbalanceadas. Un valor cercano a 1 refleja un modelo con errores mínimos.

  - ***Log-loss (Pérdida Logarítmica):***  
    Evalúa la calidad de las predicciones probabilísticas del modelo, centrándose en qué tan bien predice la probabilidad de la clase positiva. En este caso, la clase positiva corresponde a clientes propensos a cancelar su contrato, por lo que esta métrica es clave para medir el desempeño del modelo. Un valor cercano a 0 indica un modelo con alta probabilidad de prediccion correcta en la clase positiva.

- **Notas adicionales:**  
  Las métricas se eligieron cuidadosamente para ofrecer un balance entre la interpretabilidad de los resultados y su relevancia para el problema de negocio. Esto asegura que los responsables de la toma de decisiones puedan entender y utilizar los resultados del modelo de manera efectiva.

#### 6.3.4 Técnicas adicionales:

- En la preparación de los datos se utilizó **sobremuestreo (SMOTE)** y **análisis de relevancia en las predicciones para cada columna (SHAP)**.

  - **SMOTE (Synthetic Minority Over-sampling Technique):**  
    Con esta técnica se buscó otorgar balance a las clases y así evitar que los modelos aprendieran de manera desproporcionada sobre una clase en particular. El balance se logra mediante la generación de datos sintéticos, basados en los registros originales, para adicionar datos con el mismo comportamiento que la clase minoritaria.

  - **SHAP (SHapley Additive exPlanations):**  
    Esta técnica es fundamental para entender cuáles variables son determinantes en la cancelación de contratos. SHAP asigna un valor de importancia a cada variable en función de su contribución a las predicciones del modelo. Estos valores se calculan utilizando la teoría de juegos, específicamente el concepto de los valores de Shapley, que distribuyen de manera justa la importancia entre todas las variables que contribuyen al resultado.

    - **Generación de valores SHAP:**  
      Para cada predicción, SHAP descompone el impacto de cada variable, comparando cómo cambia la predicción del modelo al incluir o excluir una variable. Este análisis se realiza para todas las filas del conjunto de datos, generando una distribución de valores SHAP para cada variable. La importancia general de una variable se calcula como la media absoluta de sus valores SHAP en todo el conjunto de datos.

    - **Interpretación de los valores:**  
      Un valor SHAP alto (positivo o negativo) indica que una variable tuvo un impacto significativo en la predicción del modelo, mientras que un valor cercano a cero sugiere que la variable tuvo poco o ningún impacto. Este análisis permite identificar variables clave para estrategias de marketing o comerciales que contrarresten la pérdida de clientes.

### Notas adicionales:
1. **SHAP y explicabilidad:** Los valores SHAP no solo ayudan a entender el modelo, sino también a generar confianza en su uso al explicar sus decisiones de manera transparente.

### 6.4 Resultados

#### Comparación entre modelos

Se realizó el siguiente entrenamiento con los cuatro modelos seleccionados:

- **Datos originales:** Registros sin ninguna técnica adicional de procesamiento.
- **Datos con sobremuestreo:** Registros con un mayor número de filas que balancean la cantidad de veces que aparece cada una de las clases.
- **Datos originales con SHAP:** Registros sin sobremuestreo y con las columnas que superan el umbral (5%) de importancia definido según el valor máximo otorgado a las columnas por el método SHAP.
- **Datos con sobremuestreo y SHAP:** Registros con sobremuestreo y con las columnas que superan el umbral (5%) de importancia definido según el valor máximo otorgado a las columnas por el método SHAP.

Utilizando estos cuatro escenarios para el entrenamiento de cada modelo, se obtuvieron los siguientes resultados:

| Modelo                                         | Segundos | SP    | AUC-ROC | Precision | log_loss | Sobremuestreo |
|------------------------------------------------|----------|-------|---------|-----------|----------|---------------|
| LogisticRegression (original vs resampled)    | 65.06    | 0.0   | 0.00    | 0.04      | -0.06    | No vs Sí      |
| RandomForestClassifier (original vs resampled)| -4.14    | 0.0   | 0.01    | 0.00      | -0.05    | No vs Sí      |
| **XGBClassifier (original vs resampled)**      | **-34.42**| **0.0**| **0.05**| **0.00**  | **-0.02**| **No vs Sí**  |
| LGBMClassifier (original vs resampled)         | 41.74    | 0.0   | 0.06    | -0.03     | 0.02     | No vs Sí      |
| LogisticRegression_SHAP (original vs resampled)| 1.86     | 0.5   | 0.01    | 0.08      | -0.09    | No vs Sí      |
| **RandomForestClassifier_SHAP (original vs resampled)** | **-2.48** | **-0.5**| **-0.07** | **-0.03** | **0.04** | **No vs Sí** |
| XGBClassifier_SHAP (original vs resampled)    | -1.23    | 0.0   | 0.12    | 0.10      | -0.09    | No vs Sí      |
| LGBMClassifier_SHAP (original vs resampled)   | -2.05    | 0.0   | 0.09    | 0.08      | -0.11    | No vs Sí      |

#### Análisis de resultados

Tras analizar los valores generados por los modelos para cada una de las métricas evaluadas, se observa que los dos modelos con mejores resultados son **XGBClassifier** y **RandomForestClassifier**. La tabla muestra la diferencia de los valores generados con y sin sobremuestreo; los valores más altos (positivos o negativos) indican qué modelo realiza mejores predicciones.

Adicionalmente, se identificaron las siguientes observaciones:

- **Impacto del uso de SHAP:**  
  Los modelos ***XGBClassifier*** y ***LGBMClassifier*** son afectados negativamente por el uso de SHAP, ya que esta técnica reduce el conjunto de características y, por ende, el volumen de los datos. Esto impide que los modelos puedan aprender de manera óptima.

- **Rendimiento de RandomForestClassifier:**  
  Por el contrario, al modelo **RandomForestClassifier** le beneficia el uso combinado de SHAP y el sobremuestreo. El balance de clases que aporta el sobremuestreo evita el sesgo hacia alguna clase, mientras que la limpieza de características proporcionada por SHAP permite al modelo aprender de forma más eficiente.

En conclusión, el uso estratégico de técnicas como SHAP y el sobremuestreo puede mejorar significativamente el rendimiento de algunos modelos. Sin embargo, su impacto varía según el modelo utilizado, siendo especialmente favorable para RandomForestClassifier debido a su capacidad inherente para manejar datos complejos y equilibrados.

### 6.5 Discusión

Tras analizar los datos correspondientes a cada una de las métricas utilizadas para evaluar los resultados del modelo, e interpretar los resultados obtenidos con el modelo elegido (**RandomForestClassifier**), se llegó a las siguientes conclusiones:

1. **Análisis de columnas clave:**  
   Un análisis detallado de las columnas ***totalcharges*** y ***type_month-to-month*** revela que estas variables tienen un impacto significativo en la permanencia de los clientes. Esto sugiere que focalizar estrategias en estas áreas podría mejorar el promedio de permanencia.

2. **Entrenamiento periódico del modelo:**  
   Es importante resaltar que el modelo debe ser entrenado periódicamente con datos actualizados. Este enfoque permitirá identificar si han surgido cambios en el comportamiento de cancelación de contratos por parte de los clientes, asegurando así que las predicciones del modelo se mantengan relevantes y precisas.

En resumen, el uso del modelo **RandomForestClassifier** junto con un monitoreo constante de las variables más influyentes puede servir como una herramienta eficaz para predecir y mejorar la retención de clientes.

### 6.6 Conclusión y Recomendaciones

Como conclusiones del proyecto, se destacan los siguientes puntos:

1. **Estructura de los datos:**  
   Los datos originales cuentan con una estructura adecuada para los valores que representan. Sin embargo, para evitar complicaciones en el procesamiento de ciertas columnas y garantizar una mayor fidelidad, se sugieren las siguientes mejoras en el almacenamiento de los registros:  
   - Las columnas de tipo fecha podrían desglosarse en tres columnas separadas que representen el día, mes y año respectivamente.  
   - Las columnas que almacenan valores booleanos ("yes" y "no") podrían transformarse en valores numéricos, asignando 1 para "yes" y 0 para "no".

2. **Próximos entrenamientos del modelo:**  
   Para futuros entrenamientos del modelo elegido, se recomienda ajustar los valores de los hiperparámetros **n_estimators** y **max_depth**. Este ajuste permitirá manejar eficientemente un mayor volumen de datos que podría generarse a partir de las próximas ventas, optimizando así el rendimiento del modelo.

3. **Ventajas del modelo:**  
   El modelo **RandomForestClassifier** no solo ofrece un excelente balance entre las métricas evaluadas, sino que también presenta un bajo costo computacional. Esto lo convierte en una opción altamente competitiva y fácil de implementar. La combinación de alta precisión y tiempos de ejecución reducidos representa una ventaja estratégica para la empresa, al contar con una herramienta eficiente y confiable.

---

En resumen, implementar estas recomendaciones permitirá optimizar tanto el manejo de los datos como el rendimiento del modelo, generando un impacto positivo en la toma de decisiones empresariales.